In [1]:
import numpy as np
import numpy as np
import subprocess
import time
import copy
from utils.opt_utils import *
import os
import h5py
import pickle
from qutip import *
import utils.pulse_configs as pc
import time

In [2]:
config_dict = pc.config_dict

In [3]:
# Defining relevant values
q_ch = config_dict["qubit"]["ch"] 
A_ch = config_dict["storage_A"]["ch"]
B_ch = config_dict["storage_B"]["ch"]

wq = config_dict["qubit"]["selective_freqs"]["g"]["ge"]
wa_A = config_dict["storage_A"]["freq"]
wa_B = config_dict["storage_B"]["freq"]

In [4]:
# ========== OPTIONS ========== #
max_segs = 20
time_start = 0.0000000
time_stop = 0.000002
init_qamp = 942
init_camp = 4465

num_drives = 2
chs = [2, 4]
init_freqs = [wq, wa_A]
verbose = True
plot_opt_pulses = False
plot_pulse_viewer = False
start_split_num = 1
n_seg_jump = 1

load_pulse_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\ctrl_vqe_pi_pulse_sim_2g_interp'
load_times_file = r'opt_SNAP_times_1g.txt'
load_amps_file = r'opt_SNAP_amps_1g.txt'
use_loaded_data = False

save_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\ctrl_vqe_state_prep_1g_interp\save_data'
hdf5_name = time.strftime('%Y%m%d-%H%M%S.h5')
epochs = 2000
epochs_per_seg = 50
train_batch_size = 20
qubit_amp_scale = 4
cavity_amp_scale = 4
# ========== OPTIONS ========== #

drive_freqs = np.array(init_freqs)

qscale = []
cscale = []
for i in range(2):
    qscale.append(init_qamp * qubit_amp_scale)
    cscale.append(init_camp * cavity_amp_scale)

t_segs, amp_segs = setup_interp_segs(2 * num_drives, time_start, time_stop, init_qamp)

amp_segs[:2, :] = init_qamp
amp_segs[2:4, :] = init_camp

amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

t_segs = np.reshape(t_segs, (num_drives * 2, int(len(t_segs.flatten()) / (num_drives * 2))))

t_segs = t_segs[:, 1:-1]

for i in range(start_split_num - 1):
    t_segs, amp_segs = split_segs_flat(interp_time_wrapper(t_segs, time_start, time_stop), interp_amp_wrapper(amp_segs))
    t_segs = t_segs[:, 1:-1]
    
t_segs = interp_time_wrapper(t_segs, time_start, time_stop)

# Flips occasional bits to give it a random 
flip_bits = np.array([[False, False, True, True]])
flip_mask = np.array([np.repeat(flip_bits, np.ceil(start_split_num / 4), axis=0).flatten()[:start_split_num]])
flip_mask = np.repeat([flip_mask], num_drives * 2, axis=1)[0]

amp_segs[flip_mask] *= -1

if use_loaded_data:
    print("Loading data")
    load_pulse_times = np.loadtxt(str(os.path.join(load_pulse_dir, load_times_file)))
    load_pulse_amps = np.loadtxt(str(os.path.join(load_pulse_dir, load_amps_file)))
    
    t_segs = np.array(load_pulse_times)
    amp_segs = np.array(load_pulse_amps)

print(f't_segs: {t_segs}')

print(f'amp_segs: {amp_segs}')

# Create blank history arrays for storing optimal / past values
time_hist = []
amp_hist = []
freq_hist = []
cost_hist = []

t_segs: [[0.00000000e+00 2.41647261e-07 2.00000000e-06]
 [0.00000000e+00 1.78992545e-06 2.00000000e-06]
 [0.00000000e+00 1.64168984e-06 2.00000000e-06]
 [0.00000000e+00 1.16487155e-06 2.00000000e-06]]
amp_segs: [[ 942]
 [ 942]
 [4465]
 [4465]]


In [5]:
# Run vqe, etc
vmax = np.vectorize(max)
vmin = np.vectorize(min)

hdf5_start_index = 0
start_segs = 1
for i in range(max_segs):
    temp_amp_scale = copy.deepcopy(amp_segs)
    temp_time_scale = copy.deepcopy(t_segs[:, 1:-1])
    if np.shape(temp_amp_scale)[0] < 2:
        temp_amp_scale[:, :] = vmax(np.abs(amp_segs * qubit_amp_scale), init_qamp * np.ones(np.shape(amp_segs)) / qubit_amp_scale)
    else:
        temp_amp_scale[:2, :] = vmax(np.abs(amp_segs[:2] * qubit_amp_scale), init_qamp * np.ones(np.shape(amp_segs[:2])) / qubit_amp_scale)
        temp_amp_scale[2:, :] = vmax(np.abs(amp_segs[2:] * cavity_amp_scale), init_camp * np.ones(np.shape(amp_segs[2:])) / cavity_amp_scale)
    
    temp_time_scale = vmin(t_segs[:, 1:-1] - time_start, time_stop - t_segs[:, 1:-1])

    client_args = [chs, verbose, plot_opt_pulses, plot_pulse_viewer, time_start, time_stop, drive_freqs]
    server_args = [(len(t_segs[0]) - 2) * epochs_per_seg, train_batch_size, amp_segs, t_segs[:, 1:-1], temp_amp_scale, temp_time_scale, hdf5_name]
    
    # Save args for rl client
    cf_name = "temp_files/client_args.txt"
    with open(cf_name, "wb") as fp:
        pickle.dump(client_args, fp)
    fp.close()
    
    # Save args for rl server
    sf_name = "temp_files/server_args.txt"
    with open(sf_name, "wb") as fp:
        pickle.dump(server_args, fp)
    fp.close()
    

    os.system('cmd /c python ./exp_ctrl_vqe_run_rl_scripts.py') 
    
    # Give time for files to be updated etc
    time.sleep(1)
    
    time.sleep(10)
    opt_amps = []
    opt_times = []
    with h5py.File(os.path.join(save_dir, hdf5_name), "r") as f:
        opt_res_index = np.argmax(f[str(i + hdf5_start_index)]["evaluation"]["rewards"][()])
        opt_result = f[str(i + hdf5_start_index)]["evaluation"]["rewards"][()][opt_res_index]
        for j in range(2 * num_drives):
            opt_amps.append([f[str(i + hdf5_start_index)]["evaluation"]["actions"][f'pulse_array_{j}'][()][opt_res_index]])
        for j in range(2 * num_drives):
            opt_times.append([f[str(i + hdf5_start_index)]["evaluation"]["actions"][f'time_array_{j}'][()][opt_res_index]])
    
    # updates amplitudes and times with optimized values and reshape
    amp_segs = np.array(opt_amps)
    amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))
    
    t_segs = np.array(opt_times)
    t_segs = np.reshape(t_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))
    
    print(f'================')
    print(f'num segs: {i + start_segs} ')
    print(f'opt_amps: {amp_segs}')
    print(f'opt_times: {t_segs}')
    print(f'opt_result: {opt_result}')

    # save values to history arrays
    time_hist.append(interp_time_wrapper(t_segs, time_start, time_stop))
    amp_hist.append(interp_amp_wrapper(amp_segs))
    cost_hist.append(opt_result)
    
    np.savez(r'run_data\\' + hdf5_name[:-3] + "-" + str(i) + ".npz", time=time_hist[-1], amp=amp_hist[-1], cost=cost_hist[-1])
    
    # # Save time history
    # with h5py.File(hdf5_name, 'w'):
    #     

    for i in range(2 * n_seg_jump):
        qscale.append(init_qamp * qubit_amp_scale)
        cscale.append(init_camp * cavity_amp_scale)

    # split segments and return to start of loop
    if (i < max_segs - 1):
        for i in range(n_seg_jump):
            t_segs, amp_segs = split_segs_flat(interp_time_wrapper(t_segs, time_start, time_stop), interp_amp_wrapper(amp_segs))

num segs: 1 
opt_amps: [[ 839.11066 ]
 [  52.079468]
 [3617.961   ]
 [5387.2417  ]]
opt_times: [[2.2571570e-07]
 [1.7671507e-06]
 [1.6389487e-06]
 [1.1206164e-06]]
opt_result: [0.58826274]
num segs: 2 
opt_amps: [[ 4.6463599e+02 -4.0024914e+01]
 [-3.9431801e+00  4.3860119e+01]
 [ 3.8185510e+03  5.3862183e+02]
 [-8.1376381e+00  5.8248911e+03]]
opt_times: [[2.2940002e-07 3.1594573e-07]
 [2.2337761e-07 1.7728906e-06]
 [7.8739404e-08 1.6023093e-06]
 [1.0113355e-06 1.2557465e-06]]
opt_result: [0.66345197]
num segs: 3 
opt_amps: [[ 3.8559412e+02 -3.7224617e+00 -8.4758224e+01]
 [-5.5993210e+01  1.9892254e+01  1.1336241e+02]
 [ 1.1034283e+04  2.1879561e+03  3.9736099e+02]
 [-4.5338684e+01 -1.7700455e+02  5.9339766e+03]]
opt_times: [[1.8281588e-07 3.0057205e-07 1.0000413e-06]
 [2.4035791e-07 7.9181217e-07 1.7314493e-06]
 [6.7863652e-08 4.0863131e-07 1.6048859e-06]
 [5.7715613e-07 1.1669704e-06 1.3461945e-06]]
opt_result: [0.7013453]
num segs: 4 
opt_amps: [[ 3.1171680e+02 -1.3590846e+01 -1.8073

KeyError: "Unable to open object (object 'evaluation' doesn't exist)"

In [ ]:
f = np.load(r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\ctrl_vqe_state_prep_interp\run_data\20241001-183559-4.npz')
l_times = f['time']
l_amp = f['amp']
l_cost = f['cost']

In [ ]:
client_args = [chs, verbose, plot_opt_pulses, plot_pulse_viewer, time_start, time_stop]
l_t_segs = l_times[:, 1:-1]
l_amp_segs = l_amp[:, 1:-1]

In [ ]:
plot_input = np.append(np.append(l_amp_segs.flatten(), l_t_segs.flatten()), l_freq)

In [ ]:
exp_interp_shape_plot(np.append(np.append(l_amp_segs.flatten(), l_t_segs.flatten()), l_freq), *client_args)

In [ ]:
exp_interp_pulse_plot(np.append(np.append(l_amp_segs.flatten(), l_t_segs.flatten()), l_freq), *client_args)

In [ ]:
print(f.__dict__.keys())
f['time']
f['amp']
f['freq']